In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/nigerian-foods'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nigerian-foods/Nigerian Foods.csv


In [2]:
# EDA
import pandas as pd
import numpy as np

# Data Preprocessing 
from sklearn import preprocessing

# Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Recommender System Imps
# Content Based Filtering 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Collaborative Based Filtering 
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# To work with text data 
import re
import string

In [3]:
df = pd.read_csv("/kaggle/input/nigerian-foods/Nigerian Foods.csv")
df.head()

,Food_Name,Main_Ingredients,Description,Food_Health,Food_Class,Region,Spice_Level,Price_Range
0,Abacha,"African salad, utazi leaves, oil, fish",Cassava-based salad with spicy fish,Healthy,Traditional,South-East,Spicy,Affordable
1,Abacha and Ugba (Regular),"Abacha (Cassava Fufu), Ugba (Oilbean Seeds)",Shredded cassava fufu served with oilbean seeds.,Healthy,Traditional,Eastern,NaN,Affordable
2,Abacha and Ugba,"African salad, oil, fish, fermented oil bean s...",Cassava-based salad with spicy fish and fermen...,Healthy,Salad,South-East,Spicy,Affordable
3,Afang Soup (Spicy),"Afang Leaves, Water yam, Seafood","Spicy soup made with afang leaves, water yam, ...",Healthy,Traditional,Southern,Spicy,Affordable
4,Afang Soup,"Afang leaves, waterleaf, meat, fish",Nutritious soup made with wild spinach and wat...,Healthy,Traditional,South-South,Mild,Affordable


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Food_Name         109 non-null    object
 1   Main_Ingredients  109 non-null    object
 2   Description       109 non-null    object
 3   Food_Health       109 non-null    object
 4   Food_Class        109 non-null    object
 5   Region            109 non-null    object
 6   Spice_Level       94 non-null     object
 7   Price_Range       109 non-null    object
dtypes: object(8)
memory usage: 6.9+ KB


In [5]:
df.isnull().sum()

Food_Name            0
Main_Ingredients     0
Description          0
Food_Health          0
Food_Class           0
Region               0
Spice_Level         15
Price_Range          0
dtype: int64

## Data Proprocessing

In [6]:
df['Spice_Level'].fillna('None',inplace=True)
df.head()

/tmp/ipykernel_18/531181275.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Spice_Level'].fillna('None',inplace=True)


,Food_Name,Main_Ingredients,Description,Food_Health,Food_Class,Region,Spice_Level,Price_Range
0,Abacha,"African salad, utazi leaves, oil, fish",Cassava-based salad with spicy fish,Healthy,Traditional,South-East,Spicy,Affordable
1,Abacha and Ugba (Regular),"Abacha (Cassava Fufu), Ugba (Oilbean Seeds)",Shredded cassava fufu served with oilbean seeds.,Healthy,Traditional,Eastern,None,Affordable
2,Abacha and Ugba,"African salad, oil, fish, fermented oil bean s...",Cassava-based salad with spicy fish and fermen...,Healthy,Salad,South-East,Spicy,Affordable
3,Afang Soup (Spicy),"Afang Leaves, Water yam, Seafood","Spicy soup made with afang leaves, water yam, ...",Healthy,Traditional,Southern,Spicy,Affordable
4,Afang Soup,"Afang leaves, waterleaf, meat, fish",Nutritious soup made with wild spinach and wat...,Healthy,Traditional,South-South,Mild,Affordable


In [7]:
features = ['Main_Ingredients','Description', 'Food_Health','Food_Class','Region','Spice_Level']

In [8]:
def create_food(x):
    return x['Main_Ingredients'] + " " + x['Description'] + " " + x['Food_Health']+ " " + x['Food_Health']+ " " + x['Food_Class']+ " " + x['Region']+ " " + x['Spice_Level']

In [9]:
df['full_decs'] = df.apply(create_food, axis=1)
df

,Food_Name,Main_Ingredients,Description,Food_Health,Food_Class,Region,Spice_Level,Price_Range,full_decs
0,Abacha,"African salad, utazi leaves, oil, fish",Cassava-based salad with spicy fish,Healthy,Traditional,South-East,Spicy,Affordable,"African salad, utazi leaves, oil, fish Cassava..."
1,Abacha and Ugba (Regular),"Abacha (Cassava Fufu), Ugba (Oilbean Seeds)",Shredded cassava fufu served with oilbean seeds.,Healthy,Traditional,Eastern,None,Affordable,"Abacha (Cassava Fufu), Ugba (Oilbean Seeds) Sh..."
2,Abacha and Ugba,"African salad, oil, fish, fermented oil bean s...",Cassava-based salad with spicy fish and fermen...,Healthy,Salad,South-East,Spicy,Affordable,"African salad, oil, fish, fermented oil bean s..."
3,Afang Soup (Spicy),"Afang Leaves, Water yam, Seafood","Spicy soup made with afang leaves, water yam, ...",Healthy,Traditional,Southern,Spicy,Affordable,"Afang Leaves, Water yam, Seafood Spicy soup ma..."
4,Afang Soup,"Afang leaves, waterleaf, meat, fish",Nutritious soup made with wild spinach and wat...,Healthy,Traditional,South-South,Mild,Affordable,"Afang leaves, waterleaf, meat, fish Nutritious..."
...,...,...,...,...,...,...,...,...,...
104,Ukodo,"Yam, plantains, meat, fish",Yam and plantain pepper soup with assorted meats,Healthy,Traditional,South-East,Spicy,Affordable,"Yam, plantains, meat, fish Yam and plantain pe..."
105,Ukwa (Breadfruit),"Breadfruit, palm oil, spices","Special occasion dish made with breadfruit, a ...",Healthy,Traditional,South-East,Varies,Moderate-Expensive,"Breadfruit, palm oil, spices Special occasion ..."
106,White Soup,"Yams, Ugu (Spinach), Palm Oil","Creamy white soup made with yams, spinach, and...",Healthy,Traditional,Eastern,None,Affordable,"Yams, Ugu (Spinach), Palm Oil Creamy white sou..."
107,White Soup (Goat),Catfish (substitute with expensive protein - g...,White soup with goat meat as a pricier alterna...,Healthy,Traditional,South-East,Mild,Moderate-Expensive,Catfish (substitute with expensive protein - g...


In [10]:
df['full_decs']

0      African salad, utazi leaves, oil, fish Cassava...
1      Abacha (Cassava Fufu), Ugba (Oilbean Seeds) Sh...
2      African salad, oil, fish, fermented oil bean s...
3      Afang Leaves, Water yam, Seafood Spicy soup ma...
4      Afang leaves, waterleaf, meat, fish Nutritious...
                             ...                        
104    Yam, plantains, meat, fish Yam and plantain pe...
105    Breadfruit, palm oil, spices Special occasion ...
106    Yams, Ugu (Spinach), Palm Oil Creamy white sou...
107    Catfish (substitute with expensive protein - g...
108    Catfish, expensive starches (plantain flour), ...
Name: full_decs, Length: 109, dtype: object

In [11]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['full_decs'])

In [12]:
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [13]:
df = df.reset_index()
indices = pd.Series(df.index, index=df['Food_Name'])

In [14]:
indices

Food_Name
Abacha                         0
Abacha and Ugba (Regular)      1
Abacha and Ugba                2
Afang Soup (Spicy)             3
Afang Soup                     4
                            ... 
Ukodo                        104
Ukwa (Breadfruit)            105
White Soup                   106
White Soup (Goat)            107
White Soup (Special)         108
Length: 109, dtype: int64

In [15]:
def get_recommendations(title, cosine_sim=cosine_sim2):
    print(title)
    idx = indices[title]
    print(idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    # print(sim_scores)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 5 most similar food
    sim_scores = sim_scores[1:11]
    
    food_indices = [i[0] for i in sim_scores]
    
    # Add sim_score to the DataFrame
    df_recommendations = df.iloc[food_indices].copy()
    df_recommendations['sim_score'] = [round(score * 100, 2) for score in [i[1] for i in sim_scores]]

    return df_recommendations


In [16]:
get_recommendations('Jollof Rice')

Jollof Rice
58


,index,Food_Name,Main_Ingredients,Description,Food_Health,Food_Class,Region,Spice_Level,Price_Range,full_decs,sim_score
57,57,Jollof Rice (Spicy),"Rice, Tomatoes, Peppers, Onions, Meat (optional)",Savory tomato-based rice dish with meat or veg...,Moderately Healthy,Traditional,Nationwide,Spicy,Affordable,"Rice, Tomatoes, Peppers, Onions, Meat (optiona...",65.58
32,32,Coconut Rice,"Rice, coconut milk, vegetables, chicken",Rice cooked with coconut milk and spices,Moderately Healthy,Modern,Nationwide,Mild,Affordable,"Rice, coconut milk, vegetables, chicken Rice c...",55.00
59,59,Jollof Rice (Party),"Rice, expensive protein (assorted meats, seafo...",Party-sized Jollof rice with a wider variety o...,Moderately Healthy,Traditional,Nationwide,Medium,Expensive,"Rice, expensive protein (assorted meats, seafo...",54.43
50,50,Fried Rice,"Rice, vegetables, chicken, shrimp",Stir-fried rice with colorful vegetables and p...,Moderately Healthy,Modern,Nationwide,Mild,Affordable,"Rice, vegetables, chicken, shrimp Stir-fried r...",53.07
13,13,Ayamase Sauce,"Green peppers, onions, locust beans, spices","Spicy green pepper sauce, often paired with Of...",Moderately Healthy,Modern,South-West,Spicy,Affordable,"Green peppers, onions, locust beans, spices Sp...",53.07
76,76,Ofada Rice (Special),"Ofada rice, special palm oil, locust beans, sp...",Ofada rice dish prepared with a high-quality p...,Moderately Healthy,Traditional,South-West,Spicy,Moderate-Expensive,"Ofada rice, special palm oil, locust beans, sp...",52.30
31,31,Coconut Rice (Regular),"Rice, Coconut Milk, Spices",Savory rice dish cooked with coconut milk and ...,Healthy,Traditional,Southern,None,Affordable,"Rice, Coconut Milk, Spices Savory rice dish co...",48.99
29,29,Chinchinga,"Kebabs, vegetables, spices","Grilled skewers, a popular street food",Moderately Healthy,Snack,North,Spicy,Affordable,"Kebabs, vegetables, spices Grilled skewers, a ...",46.83
33,33,Coconut Rice (Seafood),"Rice, coconut milk, vegetables, expensive seaf...","Rice cooked with coconut milk and spices, feat...",Moderately Healthy,Modern,Nationwide,Mild,Expensive,"Rice, coconut milk, vegetables, expensive seaf...",44.23
75,75,Obe Ata,"Chicken, palm oil, tomatoes, peppers, spices",Spicy chicken stew cooked in palm oil,Moderately Healthy,Stew,South-South,Spicy,Affordable,"Chicken, palm oil, tomatoes, peppers, spices S...",44.23


In [17]:
import pickle

# Assuming cosine_sim2 is the cosine similarity matrix you want to save
with open('cosine_sim2.pkl', 'wb') as file:
    pickle.dump(cosine_sim2, file)